In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6)

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
    x_sq = x * x

    with t.stop_recording():
        y_sq = y * y
    
    z = x_sq + y_sq

grad = t.gradient(z, {'x': x, 'y': y})
print('dz/dx: ', grad['x'])
print('dz/dy: ', grad['y'])

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)
reset = True

with tf.GradientTape() as t:
    y_sq = y*y
    
    if reset:
        t.reset()
    
    z = x*x + y_sq

grad = t.gradient(z, {'x': x, 'y': y})
print('dz/dx = ', grad['x'])
print('dz/dy = ', grad['y'])

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
    y_sq = y**2

    z = x**2 + tf.stop_gradient(y_sq)

grad = t.gradient(z, {'x': x, 'y': y})
print('dz/dx: ', grad['x'])
print('dz/dy: ', grad['y'])

In [ ]:
@tf.custom_gradient
def clip_gradients(y):
    def backward(dy):
        return tf.clip_by_norm(dy, 0.5)
    
    return y, backward

v = tf.Variable(2.0)
with tf.GradientTape() as t:
    output = clip_gradients(v*v)

print(t.gradient(output, v))

In [ ]:
x0 = tf.constant(0.0)
x1 = tf.constant(0.0)

with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
    tape0.watch(x0)
    tape1.watch(x1)

    y0 = tf.math.sin(x0)
    y1 = tf.nn.sigmoid(x1)

    y = y0 + y1

    ys = tf.reduce_sum(y)

In [ ]:
tape0.gradient(ys, x0)

In [ ]:
tape1.gradient(ys, x1).numpy()

In [ ]:
x = tf.Variable(1.0)

with tf.GradientTape() as t2:
    with tf.GradientTape() as t1:
        y = x*x*x

    dy_dx = t1.gradient(y, x)
dy2_dx2 = t2.gradient(dy_dx, x)

print(dy_dx.numpy())
print(dy2_dx2.numpy())

In [ ]:
x = tf.random.normal([7,5])

layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

In [ ]:
with tf.GradientTape() as t2:
    with tf.GradientTape(watch_accessed_variables=False) as t1:
        t1.watch(x)
        y = layer(x)
        out = tf.reduce_sum(layer(x)**2)
    g1 = t1.gradient(out, x)
    g1_mag = tf.norm(g1)

dg1_mag = t2.gradient(g1_mag, layer.trainable_variables)

In [ ]:
[var for var in dg1_mag]

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)
delta = tf.Variable(0.0)

with tf.GradientTape() as tape:
    y = tf.nn.sigmoid(x+delta)

dy_dx = tape.jacobian(y, delta)

In [ ]:
print(y.shape)
print(dy_dx)

In [ ]:
plt.plot(x.numpy(), y, c='b', label='y')
plt.plot(x.numpy(), dy_dx, c='r', label='dy/dx')
plt.legend()
plt.show()

In [ ]:
x = tf.random.normal([7, 5])
layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

with tf.GradientTape(persistent=True) as tape:
    y = layer(x)

y.shape

In [ ]:
layer.kernel.shape

In [ ]:
j = tape.jacobian(y, layer.kernel)
j.shape

In [ ]:
g = tape.gradient(y, layer.kernel)
print('g.shape: ', g.shape)

In [ ]:
j_sum = tf.reduce_sum(j, axis=[0,1])
delta = tf.reduce_max(abs(g - j_sum)).numpy()

assert delta < 1e-3
print('delta: ', delta)